# LinkedIn Scraper

In [1]:
# Dependencies
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import requests
from bs4 import BeautifulSoup
import re
import time
import json
import getpass

In [2]:
# Set driver path
path = '/Users/carlos/Documents/My Working Directory/chromedriver' #ChromeDriver path
# Initialize chromedriver to driver variable
browser = webdriver.Chrome(path)

In [3]:
#Function to log into LinkedIn
def log_In():
    #Open LinkedIn login page
    browser.get('https://www.linkedin.com/login?fromSignIn=true&trk=guest_homepage-basic_nav-header-signin')

    #Ask user to enter login info
    #User_username = input('Please enter your LinkedIn username:\n')
    #User_password = input('Please enter your LinkedIn password:\n')
    User_username = getpass.getpass("Please enter your LinkedIn username:\n")
    User_password = getpass.getpass("Please enter your LinkedIn password:\n")

    #Enter login info:
    elementID = browser.find_element_by_id('username')
    elementID.send_keys(User_username)

    elementID = browser.find_element_by_id('password')
    elementID.send_keys(User_password)

    elementID.submit()#Clicks login button

In [4]:
# Calling log_In()
log_In()

Please enter your LinkedIn username:
········
Please enter your LinkedIn password:
········


In [5]:
#Function to search for a job title, and scrape the job
#information of the first page of job listings 
job_titles = []
def Search_for_job():

    #Ask user to input job type he wishes to search for (data science internships)
    jobID = input('What job do you want to search for?\n') 
    
    #Split jobID in list of strings
    which_jobID = jobID.split()
    
    # Want add '%20' in-place of whitespace to search for see_all_page link
    space = '%20'
    see_all_jobID = ''
    for i in range(len(which_jobID)):
        see_all_jobID = see_all_jobID+which_jobID[i]+space

    
    see_all_jobID = see_all_jobID[:-3] #Removes the %20 at the
    

    # see_all *Job-Title-Page*
    see_all_page = browser.get('https://www.linkedin.com/jobs/search/?keywords='+see_all_jobID)
    
    # Store in r the see_all_page link to use bs to scrape
    r = requests.get('https://www.linkedin.com/jobs/search/?keywords='+see_all_jobID)
    #Parse html using bs
    soup = BeautifulSoup(r.text, 'html.parser')

    # Using regex method, we grab all the <a> & class="base-card__full-link"
    # text in Job-title page
    regex = re.compile(r'[/]([a-z]|[A-Z])\w+')
    results = soup.find_all('a', {'class':"base-card__full-link"}, href = regex)
    
    # Iterating through our results array to grab desired links and store them
    # in job_titles
    #job_titles = []
    for i in range(len(results)-1):
        job_titles.append(results[i].attrs['href'])
    
    #return job_titles

In [6]:
# searching for 'data science internships' 
Search_for_job()

What job do you want to search for?
data science internships


In [54]:
len(job_titles) # 24 job posts on the 1st page

24

In [55]:
# The function that takes our job-title links as an input
# The input (aa) is the job_titles array
Date_Posted = []
Job_Info = []
    
def get_job_info(aa):    
    for a in aa:
        time.sleep(5) #Avoid Too Many Requests Error
        # Parse info on individual job_title page
        src = requests.get(a)
        soup2 = BeautifulSoup(src.text, 'html.parser')
        # Parse json code block
        data = soup2.find_all('script', {'type':'application/ld+json'})[0].text
        info = json.loads(data)
        # Add date job was posted to list
        Date_Posted.append(info['datePosted'][0:10])
        # Parse job_description to get rid of embedded tags
        soup3 = BeautifulSoup(info['description'])
        soup4 = BeautifulSoup(soup3.text, 'html.parser')
        # Replace <br> tags by white space
        for e in soup4.find_all('br'):
            e.replace_with(' ')
        
        #Get only text back without imbedded tags & store it in Job_Info list/array
        Job_Info.append(soup4.text)

In [56]:
# Calling the function
get_job_info(job_titles)

In [57]:
len(Job_Info) # 24 job descriptions

24

### Store scraped information into pandas data frame

In [58]:
# Dependencies
import numpy as np
import pandas as pd

In [59]:
# Store all extracted job info in Pandas dataframe
df = pd.DataFrame(np.array(Job_Info), columns=['Job Information'])
# Let's add the Date_Posted as well as a column in the dataframe
df['Job Posted on'] = Date_Posted
df.head()

,Job Information,Job Posted on
0,With over 35 nationalities and a range of back...,2022-01-21
1,Responsibilities Our Data Science and Analyti...,2022-01-14
2,GoHealth is looking for a Data Scientist Inter...,2022-01-15
3,Join our awesome team as a Data Science Intern...,2022-01-19
4,To Apply Please Visit https://lever.careerst...,2022-01-20


In [60]:
df.tail()

,Job Information,Job Posted on
19,Job Description As a science-based healthcare...,2022-01-21
20,"Every month, millions of people across the wor...",2022-01-17
21,Date Posted: 2021-10-28-07:00 Country: Un...,2021-12-19
22,"With over a million customers trusting PayJoy,...",2022-01-26
23,We are looking for Machine Learning Interns to...,2022-01-07
